<a href="https://colab.research.google.com/github/ruemee/Zindi_Competitions/blob/main/InstaDeep_Enzyme_Classification_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
np.random.seed(123)
warnings.filterwarnings('ignore')
%matplotlib inline

In [51]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [52]:
link = 'https://drive.google.com/file/d/10jEwUYUedYzx_3ihN-nSioKLJKv4KeXR/view?usp=sharing' # The shareable link


In [53]:
# to get the id part of the file 
id = link.split("/")[-2] 
  
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('Train.csv')   
  
df = pd.read_csv('Train.csv') 
df.head()

,SEQUENCE_ID,SEQUENCE,CREATURE,LABEL
0,00011UBJ,MPWQELSITVPHEYVEPISYLFGRYGKGVSTELAGNGQVLLRTYLT...,creature9,class11
1,0001HZ3P,MGKLVIGSRGSELALWQANHIKERLKKECLIESEIQIVKTKGDKIL...,creature3,class13
2,0003NJDO,MFIVAVLMLAFLIFVHELGHFTIARICGVKVEVFSIGFGKKLCFFK...,creature3,class16
3,0003XVE3,MKNTSLSTTLTMNDTIAAIATPLGKGAISIIKISGHNALNILKQLT...,creature3,class12
4,00049BS0,MGNRLTKIYTKTGDSGLTGLADGSRTEKDSLRIFVIGDVDELNSLL...,creature8,class13


In [54]:
id = link.split("/")[-2] 
  
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('Test.csv')   
  
df_test = pd.read_csv('Test.csv') 
df_test.head()

,SEQUENCE_ID,SEQUENCE,CREATURE,LABEL
0,00011UBJ,MPWQELSITVPHEYVEPISYLFGRYGKGVSTELAGNGQVLLRTYLT...,creature9,class11
1,0001HZ3P,MGKLVIGSRGSELALWQANHIKERLKKECLIESEIQIVKTKGDKIL...,creature3,class13
2,0003NJDO,MFIVAVLMLAFLIFVHELGHFTIARICGVKVEVFSIGFGKKLCFFK...,creature3,class16
3,0003XVE3,MKNTSLSTTLTMNDTIAAIATPLGKGAISIIKISGHNALNILKQLT...,creature3,class12
4,00049BS0,MGNRLTKIYTKTGDSGLTGLADGSRTEKDSLRIFVIGDVDELNSLL...,creature8,class13


In [ ]:
df.dtypes

SEQUENCE_ID    object
SEQUENCE       object
CREATURE       object
LABEL          object
dtype: object

In [55]:
df_test.dtypes

SEQUENCE_ID    object
SEQUENCE       object
CREATURE       object
LABEL          object
dtype: object

In [ ]:
df.isnull().sum()

SEQUENCE_ID    0
SEQUENCE       0
CREATURE       0
LABEL          0
dtype: int64

In [56]:
df_test.isnull().sum()

SEQUENCE_ID    0
SEQUENCE       0
CREATURE       0
LABEL          0
dtype: int64

In [ ]:
df["CREATURE"].value_counts()

creature8    140738
creature3    138450
creature4    133900
creature9    111984
creature0     90240
creature1     84968
creature2     80405
creature5     78092
Name: CREATURE, dtype: int64

In [59]:
df['SEQUENCE_ID'] = df['SEQUENCE_ID'].astype('category')
df['LABEL'] = df['LABEL'].astype('category')
df['SEQUENCE'] = df['SEQUENCE'].astype('category')
df['CREATURE'] = df['CREATURE'].astype('category')


In [60]:
df["LABEL"] = df["LABEL"].cat.codes
df["CREATURE"] = df["CREATURE"].cat.codes

In [61]:
df["SEQUENCE"] = df["SEQUENCE"].cat.codes
df["SEQUENCE_ID"] = df["SEQUENCE_ID"].cat.codes

In [ ]:
df = df.astype('int')

In [62]:
df.head()

,SEQUENCE_ID,SEQUENCE,CREATURE,LABEL
0,0,447551,7,3
1,1,187635,3,5
2,2,169603,3,8
3,3,289289,3,4
4,4,191183,6,5


In [63]:
from sklearn.model_selection import train_test_split

columns = ['SEQUENCE_ID', 'SEQUENCE', 'CREATURE']


all_X = df[columns]
all_y = df['LABEL']

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.2,random_state=0)
train_X.shape

(687021, 3)

In [64]:
from sklearn.model_selection import train_test_split

X = df.drop(['LABEL'],axis=1).values   # independant features
y = df['LABEL'].values					# dependant variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [65]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_cl = xgb.XGBClassifier(n_estimators=100, learning_rate=0.5, max_delta_step=5)
xgb_cl.fit(X_train, y_train)

prediction = xgb_cl.predict(X_test)
pred =accuracy_score(y_test ,prediction)

In [66]:
print(pred)

0.5661527821395428


In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.89      0.41      0.56      3694
           1       0.79      0.50      0.61     11859
           2       0.79      0.36      0.50     11754
           3       0.69      0.41      0.51     20458
           4       0.77      0.39      0.51     12449
           5       0.78      0.41      0.54      9856
           6       0.86      0.42      0.57      5928
           7       0.90      0.56      0.69      4624
           8       0.79      0.40      0.54      9334
           9       0.84      0.47      0.60      4901
          10       0.90      0.45      0.60      4289
          11       0.45      0.95      0.61     84448
          12       0.76      0.41      0.54     17538
          13       0.87      0.41      0.55     10889
          14       0.81      0.44      0.57     10649
          15       0.73      0.44      0.55     12178
          16       0.66      0.38      0.48     15240
          17       0.70    

In [69]:
# Get the predicted result for the test Data
df_test.LABEL = xgb_cl.predict(pred)

ValueError: ignored

In [71]:
submission = pd.DataFrame({"SEQUENCE_ID": df_test["SEQUENCE_ID"] ,

"LABEL": df_test.LABEL})

In [72]:
submission.sample(5)

,SEQUENCE_ID,LABEL
779761,WOUOJJP3,class5
762047,VYFQ5WI2,class19
296860,CGWZDDHC,class19
444597,IO4D1JSS,class10
595747,P00U5GWJ,class1


In [74]:
submission.to_csv('first_submission.csv')

In [ ]:
res = pd.DataFrame(prediction)
res.columns = ["prediction"]
res.to_csv("prediction_results.csv")

AttributeError: ignored